In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

IMG_SIZE_INCEPTION = 299
BATCH_SIZE_INCEPTION = 16

train_tf_inception = transforms.Compose([
    transforms.Resize((IMG_SIZE_INCEPTION, IMG_SIZE_INCEPTION)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_tf_inception = transforms.Compose([
    transforms.Resize((IMG_SIZE_INCEPTION, IMG_SIZE_INCEPTION)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset_inception = datasets.ImageFolder(
    root=f"{DATA_DIR}/train",
    transform=train_tf_inception
)

val_dataset_inception = datasets.ImageFolder(
    root=f"{DATA_DIR}/val",
    transform=val_tf_inception
)

test_dataset_inception = datasets.ImageFolder(
    root=f"{DATA_DIR}/test",
    transform=val_tf_inception
)

train_loader_inception = DataLoader(
    train_dataset_inception,
    batch_size=BATCH_SIZE_INCEPTION,
    shuffle=True,
    num_workers=NUM_WORKERS
)

val_loader_inception = DataLoader(
    val_dataset_inception,
    batch_size=BATCH_SIZE_INCEPTION,
    shuffle=False,
    num_workers=NUM_WORKERS
)

test_loader_inception = DataLoader(
    test_dataset_inception,
    batch_size=BATCH_SIZE_INCEPTION,
    shuffle=False,
    num_workers=NUM_WORKERS
)

print("Inception classes:", train_dataset_inception.classes)
print("Number of classes:", len(train_dataset_inception.classes))

In [ ]:
import torch.nn as nn
import torch.optim as optim


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

inception = models.inception_v3(
    weights=models.Inception_V3_Weights.IMAGENET1K_V1,
    aux_logits=True
)

for p in inception.parameters():
    p.requires_grad = False

for name, p in inception.named_parameters():
    if any(k in name for k in ["Mixed_6", "Mixed_7"]):
        p.requires_grad = True

num_classes = len(train_dataset_inception.classes)

# Replace classifier
inception.fc = nn.Sequential(
    nn.Linear(inception.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, num_classes)
)

for p in inception.fc.parameters():
    p.requires_grad = True

inception = inception.to(DEVICE)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, inception.parameters()),
    lr=2e-4,          
    weight_decay=1e-4
)

scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=18
)

print("InceptionV3 ready")

In [ ]:
EPOCHS = 18
best_val_acc = 0.0

for epoch in range(EPOCHS):

    # -------- Training --------
    inception.train()
    train_correct = 0
    train_total = 0
    train_loss = 0.0

    for images, labels in train_loader_inception:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()

        outputs, aux_outputs = inception(images)

        loss_main = criterion(outputs, labels)
        loss_aux  = criterion(aux_outputs, labels)
        loss = loss_main + 0.4 * loss_aux

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * labels.size(0)
        preds = outputs.argmax(1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

    train_acc = train_correct / train_total
    train_loss /= train_total

    # -------- Validation --------
    inception.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader_inception:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            outputs = inception(images)  
            preds = outputs.argmax(1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total

    scheduler.step()

    print(
        f"[Inception] Epoch {epoch+1}/{EPOCHS} | "
        f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}"
    )

    # -------- Save Best --------
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(inception.state_dict(), "inception_best.pth")


In [ ]:
inception.load_state_dict(
    torch.load("inception_best.pth", map_location=DEVICE)
)
inception.eval()
print("InceptionV3 weights loaded")

In [ ]:
import torch
import numpy as np

inception.eval()

criterion = torch.nn.CrossEntropyLoss()

test_correct = 0
test_total = 0
test_loss = 0.0

inception_all_preds = []
inception_all_labels = []

with torch.no_grad():
    for images, labels in test_loader_inception:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = inception(images)
        loss = criterion(outputs, labels)

        preds = outputs.argmax(1)

        test_loss += loss.item() * labels.size(0)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

        inception_all_preds.extend(preds.cpu().numpy())
        inception_all_labels.extend(labels.cpu().numpy())

test_acc = test_correct / test_total
test_loss /= test_total

print(f"InceptionV3 Test Accuracy: {test_acc:.4f}")
print(f"InceptionV3 Test Loss: {test_loss:.4f}")

inception_test_acc = test_acc

In [ ]:
print("InceptionV3 — Classification Report (Test Set)")
print(
    classification_report(
        inception_all_labels,
        inception_all_preds,
        target_names=class_names,
        digits=4
    )
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

class_names = train_dataset_inception.classes

cm = confusion_matrix(inception_all_labels, inception_all_preds)

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm,
    cmap="Blues",
    xticklabels=class_names,
    yticklabels=class_names,
    annot=False
)

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("InceptionV3 — Confusion Matrix (Test Set)")
plt.tight_layout()
plt.show()

In [ ]:
import json

with open("class_names.json", "w") as f:
    json.dump(train_dataset_inception.classes, f)

print("Saved class_names.json")